In [134]:
using LinearAlgebra
using Statistics
using DelimitedFiles
using Plots

## Process the data dumped out of `L4DC Controller Synthesis.ipynb`

All of the data matrices are indexed by `[b, m]`,
where `b = 1, 2, 3, ..., 7` and `m = 1, 2, 3, ..., 50`
are the different bootstrap index and sample index respectively.
For instance,

* `Jstars[5, 20]` is the cost of the `20`th independent run of the MPC loop with `N = 1000` under the optimal control scheme
* `xrobBs[3, 10]` is the `||x[0 : H-1]||` of the `10`th independent run of MPC loop with `N = 100` under the robust bootstrap-epsilon control scheme




In [135]:
Jstars = readdlm("Jstars.txt", '\t', Float64)
Jstars[Jstars .>= 10] .= Inf

Jnoms = readdlm("Jnoms.txt", '\t', Float64)
Jnoms[Jnoms .>= 10] .= Inf

JrobBs = readdlm("JrobBs.txt", '\t', Float64)
JrobBs[JrobBs .>= 10] .= Inf

JrobTs = readdlm("JrobTs.txt", '\t', Float64)
JrobTs[JrobTs .>= 10] .= Inf

gammaBs = readdlm("gammaBs.txt", '\t', Float64)

gammaTs = readdlm("gammaTs.txt", '\t', Float64)

normxstars = readdlm("normxstars.txt", '\t', Float64)
normxstars[normxstars .>= 1000] .= Inf
normustars = readdlm("normustars.txt", '\t', Float64)
normustars[normustars .>= 100] .= Inf

normxnoms = readdlm("normxnoms.txt", '\t', Float64)
normxnoms[normxnoms .>= 1000] .= Inf
normunoms = readdlm("normunoms.txt", '\t', Float64)
normunoms[normunoms .>= 100] .= Inf

normxrobBs = readdlm("normxrobBs.txt", '\t', Float64)
normxrobBs[normxrobBs .>= 1000] .= Inf
normurobBs = readdlm("normurobBs.txt", '\t', Float64)
normurobBs[normurobBs .>= 100] .= Inf

normxrobTs = readdlm("normxrobTs.txt", '\t', Float64)
normxrobTs[normxrobTs .>= 1000] .= Inf
normurobTs = readdlm("normurobTs.txt", '\t', Float64)
normurobTs[normurobTs .>= 100] .= Inf

;

In [136]:
# The particular values of N that were used
# Should be in intervals of 10^1.0, 10^1.5, ...
# But the resolution of the "L4DC Sample Complexity".ipynb
# was in units of 10.
Ns = [10; 30; 100; 316; 1000; 3160; 10000;]

Bmax = length(Ns)

Jstarsmed = median(Jstars, dims=2)
JstarsQ1 = [quantile(Jstars[b,:], 0.25) for b = 1:Bmax]
JstarsQ3 = [quantile(Jstars[b,:], 0.75) for b = 1:Bmax]

JrobBsmed = median(JrobBs, dims=2)
JrobBsQ1 = [quantile(JrobBs[b,:], 0.25) for b = 1:Bmax]
JrobBsQ3 = [quantile(JrobBs[b,:], 0.75) for b = 1:Bmax]

JrobTsmed = median(JrobTs, dims=2)
JrobTsQ1 = [quantile(JrobTs[b,:], 0.25) for b = 1:Bmax]
JrobTsQ3 = [quantile(JrobTs[b,:], 0.75) for b = 1:Bmax]
;

In [137]:
plot(1:Bmax, Jstarsmed,
    ribbons=(Jstarsmed - JstarsQ1, JstarsQ3 - Jstarsmed),
    fillalpha=0.2,
    xticks=(1:10, ["1.0"; "1.5"; "2.0"; "2.5"; "3.0"; "3.5"; "4.0";]),
    ylims=(1.0,3.0),
    label="Optimal",
    xlabel="Number of Samples (log10 N)",
    ylabel="LQR Cost",
    xtickfontsize=18,
    ytickfontsize=18,
    legendfontsize=18,
    xguidefontsize=18,
    yguidefontsize=18,
        )
plot!(1:Bmax, JrobBsmed,
    ribbons=(JrobBsmed - JrobBsQ1, JrobBsQ3 - JrobBsmed),
    fillalpha=0.2,
    label="Boot ε")

plot!(1:Bmax, JrobTsmed,
    ribbons=(JrobTsmed - JrobTsQ1, JrobTsQ3 - JrobTsmed),
    fillalpha=0.2,
    label="True ε")

savefig("lqr-costs.png")


In [138]:
xstarsmed = median(normxstars, dims=2)
xstarsQ1 = [quantile(normxstars[b,:], 0.25) for b = 1:Bmax]
xstarsQ3 = [quantile(normxstars[b,:], 0.75) for b = 1:Bmax]

xrobBsmed = median(normxrobBs, dims=2)
xrobBsQ1 = [quantile(normxrobBs[b,:], 0.25) for b = 1:Bmax]
xrobBsQ3 = [quantile(normxrobBs[b,:], 0.75) for b = 1:Bmax]

xrobTsmed = median(normxrobTs, dims=2)
xrobTsQ1 = [quantile(normxrobTs[b,:], 0.25) for b = 1:Bmax]
xrobTsQ3 = [quantile(normxrobTs[b,:], 0.75) for b = 1:Bmax]
;

In [139]:
plot(1:Bmax, xstarsmed,
    ribbons=(xstarsmed - xstarsQ1, xstarsQ3 - xstarsmed),
    fillalpha=0.2,
    ylims=(40,100),
    xticks=(1:10, ["1.0"; "1.5"; "2.0"; "2.5"; "3.0"; "3.5"; "4.0";]),
    label="Optimal x[0:H-1]",
    xlabel="Number of Samples (log10 N)",
    ylabel="Norm",
    xtickfontsize=18,
    ytickfontsize=18,
    legendfontsize=18,
    xguidefontsize=18,
    yguidefontsize=18,
)

plot!(1:Bmax, xrobBsmed,
    ribbons=(xrobBsmed - xrobBsQ1, xrobBsQ3 - xrobBsmed),
    fillalpha=0.2,
    label="Boot ε x[0:H-1]")

plot!(1:Bmax, xrobTsmed,
    ribbons=(xrobTsmed - xrobTsQ1, xrobTsQ3 - xrobTsmed),
    fillalpha=0.2,
    label="True ε x[0:H-1]")

savefig("xnorms.png")

In [140]:
ustarsmed = median(normustars, dims=2)
ustarsQ1 = [quantile(normustars[b,:], 0.25) for b = 1:Bmax]
ustarsQ3 = [quantile(normustars[b,:], 0.75) for b = 1:Bmax]

urobBsmed = median(normurobBs, dims=2)
urobBsQ1 = [quantile(normurobBs[b,:], 0.25) for b = 1:Bmax]
urobBsQ3 = [quantile(normurobBs[b,:], 0.75) for b = 1:Bmax]

urobTsmed = median(normurobTs, dims=2)
urobTsQ1 = [quantile(normurobTs[b,:], 0.25) for b = 1:Bmax]
urobTsQ3 = [quantile(normurobTs[b,:], 0.75) for b = 1:Bmax]
;

In [141]:
plot(1:Bmax, ustarsmed,
    ribbons=(ustarsmed - ustarsQ1, ustarsQ3 - ustarsmed),
    fillalpha=0.2,
    ylims=(2,10),
    xticks=(1:10, ["1.0"; "1.5"; "2.0"; "2.5"; "3.0"; "3.5"; "4.0";]),
    label="Optimal u[0:H-1]",
    xlabel="Number of Samples (log10 N)",
    ylabel="Norm",
    xtickfontsize=18,
    ytickfontsize=18,
    legendfontsize=18,
    xguidefontsize=18,
    yguidefontsize=18,
)

plot!(1:Bmax, urobBsmed,
    ribbons=(urobBsmed - urobBsQ1, urobBsQ3 - urobBsmed),
    fillalpha=0.2,
    label="Boot ε u[0:H-1]")

plot!(1:Bmax, urobTsmed,
    ribbons=(urobTsmed - urobTsQ1, urobTsQ3 - urobTsmed),
    fillalpha=0.2,
    label="True ε u[0:H-1]")

savefig("unorms.png")